# Calculate SLA 

In [1]:
import numpy as np
import pandas as pd

import holidays
import business_calendar as bc
import sla_checker as sch
import re
import datetime as dt

In [2]:
df = pd.read_excel('sample & SLA.xlsx')
df.head()

,SRIDentification,TT_Created_date,TiketCloseTime,SLA
0,2101010000623771,2021-01-01 16:47:38.000,2021-01-01 19:44:05.000,2D
1,2101010000154762,2021-01-01 07:52:09.000,2021-01-01 18:58:25.000,48 H
2,2101010000284926,2021-01-01 12:34:30.000,2021-01-01 13:38:04.000,2 H
3,2101010000759068,2021-01-01 18:24:22.000,2021-01-01 18:56:40.000,1D
4,2101010000163250,2021-01-01 08:47:43.000,2021-01-03 21:07:43.000,48WH


## Re-Stracure data values and Wrangling

In [3]:
df.SLA.replace({'On Spot':'1H On Spot', 'Not Escalated': '24H Not Escalated'}, inplace = True)

In [4]:
df.SLA.value_counts()

48 H                                                          5130
5 H                                                           1159
SLA 2 H from 8AM till 9 PM except Friday from 2PM till 9PM     806
2D                                                             653
1H                                                             629
48WH                                                           333
72H                                                            324
1D                                                             253
2 H                                                            171
24 H                                                           165
1H On Spot                                                     117
1 H ( From 9 AM till 10 PM )                                   101
2H                                                              48
2WH ?working hours from 8am to 10 pm                            37
4WD                                                           

In [5]:
def extract_numbers(txt):
    return re.findall(r'\d+', txt)[0]

def extract_flags(txt):
    return txt[:4]

def remove_numbers(txt):
    return ''.join(i for i in txt if not i.isdigit())

def remove_spaces(txt):
    return "".join(txt.split())

def extraxt_opening(txt):
    
    if len(re.findall(r'\d+', txt)) > 1: 
        return re.findall(r'\d+', txt)[1]
    else:
        return None

def extraxt_ending(txt):
    
    if len(re.findall(r'\d+', txt)) > 2: 
        return re.findall(r'\d+', txt)[2]
    else:
        return None

In [6]:
df.SLA.replace('SLA 2 H from 8AM till 9 PM except Friday from 2PM till 9PM','2 H from 8AM till 9 PM except Friday from 2PM till 9PM', inplace = True)

In [7]:
df['sla_time'] = [extract_numbers(x) for x in df.SLA.values]

In [8]:
df['sla_flag'] = [extract_flags(x) for x in df.SLA.values]

In [9]:
df['sla_flag'] = [remove_numbers(x) for x in df.sla_flag.values]

In [10]:
df['sla_flag'] = [remove_spaces(x) for x in df.sla_flag.values]

In [11]:
df['sla_flag'].replace('HO','H', inplace=True)

In [12]:
df['opening_hours'] = [extraxt_opening(x) for x in df.SLA.values]
df['ending_hours'] = [extraxt_ending(x) for x in df.SLA.values]


df['opening_hours'] = df['opening_hours'].replace({'8':'08:00', '9':'09:00'})
df['opening_hours'] = pd.to_datetime(df['opening_hours'],format= '%H:%M').dt.time

df['ending_hours'] = df['ending_hours'].replace({'9':'21:00', '10':'22:00'})
df['ending_hours'] = pd.to_datetime(df['ending_hours'],format= '%H:%M').dt.time

In [13]:
df.sla_time = df.sla_time.astype('int')

In [14]:
for i, v in enumerate(df.sla_flag.values):
    if v == 'D' or v == 'WD':
        df.iloc[i, df.columns.get_loc('sla_time')] = df.iloc[i, df.columns.get_loc('sla_time')] * 24
        
df['sla_flag'] = df['sla_flag'].replace({'D':'H', 'WD':'WH'})

In [15]:
df.head()

,SRIDentification,TT_Created_date,TiketCloseTime,SLA,sla_time,sla_flag,opening_hours,ending_hours
0,2101010000623771,2021-01-01 16:47:38.000,2021-01-01 19:44:05.000,2D,48,H,NaT,NaT
1,2101010000154762,2021-01-01 07:52:09.000,2021-01-01 18:58:25.000,48 H,48,H,NaT,NaT
2,2101010000284926,2021-01-01 12:34:30.000,2021-01-01 13:38:04.000,2 H,2,H,NaT,NaT
3,2101010000759068,2021-01-01 18:24:22.000,2021-01-01 18:56:40.000,1D,24,H,NaT,NaT
4,2101010000163250,2021-01-01 08:47:43.000,2021-01-03 21:07:43.000,48WH,48,WH,NaT,NaT


## SLA Checker

In [ ]:
from sla_calculator import SLA_Calculator

sla_calc = SLA_Calculator()

for i in range(len(df))
    sla_time = sla_calc.calculate(start_time="2019-12-10T01:02:03Z",
                                  open_hour=9,
                                  close_hour=17,
                                  country_name="US",
                                  sla_in_hours=4)
print(sla_time.to_iso8601_string())

In [16]:
len(df)

10000